In [2]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
import torch

In [ ]:
bert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
bert = DistilBertForMaskedLM.from_pretrained('distilbert-base-cased')

In [ ]:
#nlp pipeline 
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
import numpy as np
def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()


def pred_top_cbow(sentences: list[str]) -> list[tuple[list[str], list[float]]]:
    string = " ".join(sentences)
    encoded_text = bert_tokenizer.encode(string, add_special_tokens=True)
    mask_token_idx = (np.array(encoded_text) == bert_tokenizer.mask_token_id).nonzero()[0]
    input = torch.tensor([encoded_text])

    with torch.no_grad():
        output = bert(input)[0]
           
    ret = []
    for i in mask_token_idx:
        v, top_preds = torch.topk(output[0, i], 500)
        words = bert_tokenizer.convert_ids_to_tokens(top_preds)
        ret.append((words, v.numpy()))

    return ret

cache = {}
def probs(s):
    result = pred_top_cbow(s)

    print(len(result))
    # cache[s[0]] = [(a,exp_normalize(b)) for a,b in result]
    # print(cache[s[0]])
    # cache[s[0]] = cache[s[0]][:1]
    # print("abcdefghi")
    # print(cache[s[0]])


text = 'This is a good book .'
preprocessed_words = nlp(text)

## Text generator
mask = bert_tokenizer.mask_token

## Sentance pertubator
words = np.array([x.text for x in preprocessed_words], dtype='|U80')

n_words = len(words)

word_matrix = np.tile(words, (n_words, 1))
np.fill_diagonal(word_matrix, mask)
masked_sentences = [" ".join(x) for x in word_matrix]

for sentence in masked_sentences:
    probs([sentence])



In [3]:
bert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
bert = DistilBertForMaskedLM.from_pretrained('distilbert-base-cased')

import spacy
nlp = spacy.load('en_core_web_sm')


text_to_explain = "This is a good book."
text_to_explain = nlp(text_to_explain)


In [20]:
import numpy as np

def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

# Textgenerator unmask
def unmask(s):
    encoded_text = bert_tokenizer.encode(s, add_special_tokens=True)
    mask_token_idx = (np.array(encoded_text) == bert_tokenizer.mask_token_id).nonzero()[0]
    input = torch.tensor([encoded_text])

    with torch.no_grad():
        output = bert(input)[0]
           
    ret = []
    for i in mask_token_idx:
        v, top_preds = torch.topk(output[0, i], 500)
        words = bert_tokenizer.convert_ids_to_tokens(top_preds)
        ret.append((words, v.numpy()))

    return ret


# Setence prob func
test = {}
def prob(s):
    result = unmask(s)
    print(len(result))
    test[s] = [(a, exp_normalize(b)) for a, b in result]

    return test[s]
# Sentence pertub init

words_in_text = [word.text for word in text_to_explain]
words_in_np = np.array(words_in_text, '|U80')

for i in range(len(words_in_text)):
    a = words_in_np.copy()
    a[i] = bert_tokenizer.mask_token

    s_a = " ".join(a)
    w, p = prob(s_a)[0]

    print(min(0.5, dict(zip(w, p)).get(words_in_text[i], 0.01)))


print(words_in_text)
a = words_in_text.copy()
a[4] = bert_tokenizer.mask_token

s = ' '.join(a)
words, p = prob(s)[0]
a[4] = np.random.choice(words, p=p)

1
0.031633068
1
0.5
1
0.5
1
0.002982501
1
0.021095624
1
0.35414582
['This', 'is', 'a', 'good', 'book', '.']
['This', 'is', 'a', 'good', 'book', '.']
0


IndexError: list index out of range